In [ ]:
import pandas as pd
import requests
import json
import time
from typing import List, Dict
import subprocess
import signal
import sys
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'llm_evaluation_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)

# Constants
OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODELS = [
    "hf.co/bartowski/Qwen2.5-7B-Instruct-GGUF:Q5_K_L",
    "hf.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF:Q5_K_L",
    "hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF:F16",
    "hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF:F16",
    "hf.co/bartowski/Qwen2.5-14B-Instruct-GGUF:Q2_K",
    "hf.co/bartowski/Qwen2.5-3B-Instruct-GGUF:F16",
    "hf.co/bartowski/Qwen2.5-1.5B-Instruct-GGUF:F16",
    "hf.co/bartowski/Phi-3-medium-128k-instruct-GGUF:Q3_K_S",
    "hf.co/RichardErkhov/princeton-nlp_-_gemma-2-9b-it-SimPO-gguf:Q4_K_M",
    "hf.co/mradermacher/gemma-2-9b-it-SimPO-GGUF:F16",
    "hf.co/RichardErkhov/princeton-nlp_-_gemma-2-9b-it-SimPO-gguf:Q8_0",
    "hf.co/DevQuasar/ai21labs.AI21-Jamba-1.5-Mini-GGUF:Q4_K_M"
]

def load_questions() -> pd.DataFrame:
    """Load questions from CSV file."""
    try:
        df = pd.read_csv('/home/ubuntu/quantumLeap/quantumEval_claude.csv')
        logging.info(f"Successfully loaded {len(df)} questions from CSV")
        return df
    except Exception as e:
        logging.error(f"Error loading questions: {e}")
        sys.exit(1)

def start_ollama_model(model_name: str) -> subprocess.Popen:
    """Start an Ollama model."""
    try:
        process = subprocess.Popen(
            ['ollama', 'run', model_name],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        time.sleep(10)  # Wait for model to initialize
        logging.info(f"Started model: {model_name}")
        return process
    except Exception as e:
        logging.error(f"Error starting model {model_name}: {e}")
        return None

def stop_ollama_model(process: subprocess.Popen):
    """Stop the Ollama model process."""
    if process:
        process.send_signal(signal.SIGTERM)
        process.wait()
        logging.info("Stopped Ollama model")

def query_model(question: str, model_name: str) -> str:
    """Query the model and get response."""
    prompt = f"Please answer the following question about Clotaire Rapaille's Culture Code methodology:\n\n{question}\n\nProvide a clear and concise answer based on Rapaille's work."
    
    payload = {
        "model": model_name,
        "prompt": prompt,
        "stream": False
    }
    
    try:
        response = requests.post(OLLAMA_API_URL, json=payload)
        response.raise_for_status()
        return response.json()['response']
    except Exception as e:
        logging.error(f"Error querying model {model_name}: {e}")
        return f"ERROR: {str(e)}"

def evaluate_models(questions_df: pd.DataFrame) -> pd.DataFrame:
    """Evaluate all models on all questions."""
    results = []
    
    for model in MODELS:
        logging.info(f"Starting evaluation for model: {model}")
        
        # Start the model
        process = start_ollama_model(model)
        if not process:
            continue
            
        try:
            for _, row in questions_df.iterrows():
                logging.info(f"Processing Question {row['Question Number']}")
                
                response = query_model(row['Question Text'], model)
                
                result = {
                    'Question Number': row['Question Number'],
                    'Attribute': row['Attribute'],
                    'Expertise Level': row['Expertise Level'],
                    'Question Text': row['Question Text'],
                    'LLM Name': model,
                    'Model Response': response,
                    'Score': '',  # To be filled later
                    'Explanation': ''  # To be filled later
                }
                results.append(result)
                
                # Save intermediate results
                pd.DataFrame(results).to_csv(
                    f'results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv',
                    index=False
                )
                
                time.sleep(2)  # Brief pause between questions
                
        except Exception as e:
            logging.error(f"Error during evaluation of model {model}: {e}")
        finally:
            stop_ollama_model(process)
            time.sleep(5)  # Wait before starting next model
    
    return pd.DataFrame(results)

def main():
    """Main execution function."""
    try:
        # Load questions
        questions_df = load_questions()
        
        # Run evaluations
        results_df = evaluate_models(questions_df)
        
        # Save final results
        output_file = f'final_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
        results_df.to_csv(output_file, index=False)
        logging.info(f"Results saved to {output_file}")
        
    except Exception as e:
        logging.error(f"Error in main execution: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()